In [2]:
import pennylane as qml
from pennylane import numpy as pnp
import os, json
import numpy as np
from collections import Counter
import pandas as pd
from datetime import datetime
from math import log2, ceil
from susy_qm import calculate_Hamiltonian, ansatze

import git
repo = git.Repo('.', search_parent_directories=True)
repo_path = repo.working_tree_dir

In [4]:
##############################################################################
# 1. Bosonic q, p in truncated HO basis
##############################################################################

def create_matrix(cutoff, kind, m=1.0):
    """Return q or p in HO Fock basis up to 'cutoff'."""
    mat = np.zeros((cutoff, cutoff), dtype=np.complex128)
    for i in range(cutoff):
        if i > 0:
            if kind == "q":
                mat[i, i-1] = (1.0/np.sqrt(2.0*m)) * np.sqrt(i)
            elif kind == "p":
                mat[i, i-1] = 1j*np.sqrt(m/2.0) * np.sqrt(i)
        if i < cutoff - 1:
            if kind == "q":
                mat[i, i+1] = (1.0/np.sqrt(2.0*m)) * np.sqrt(i+1)
            elif kind == "p":
                mat[i, i+1] = -1j*np.sqrt(m/2.0) * np.sqrt(i+1)
    return mat

##############################################################################
# 2. Dense bosonic blocks only (size cutoff x cutoff)
##############################################################################

def bosonic_blocks(cutoff, potential, m=1.0, g=1.0, u=1.0):
    """
    Build bosonic dense pieces:
      A_b = p^2 + W'(q)^2
      B_b = W''(q)
    in the boson Fock basis.
    """
    q = create_matrix(cutoff, "q", m=m)
    p = create_matrix(cutoff, "p", m=m)

    q2 = q @ q
    q3 = q2 @ q
    I_b = np.eye(cutoff, dtype=np.complex128)

    if potential == "QHO":
        W_prime = m * q
        W_pp    = m * I_b
    elif potential == "AHO":
        W_prime = m * q + g * q3
        W_pp    = m * I_b + 3.0 * g * q2
    elif potential == "DW":
        W_prime = m * q + g * q2 + g * (u**2) * I_b
        W_pp    = m * I_b + 2.0 * g * q
    else:
        raise ValueError("potential must be 'QHO', 'AHO', or 'DW'.")

    p2 = p @ p
    A_b = p2 + (W_prime @ W_prime)
    B_b = W_pp

    # clean tiny numerical noise
    A_b[np.abs(A_b) < 1e-12] = 0.0
    B_b[np.abs(B_b) < 1e-12] = 0.0
    return A_b, B_b

##############################################################################
# 3. Boson dense -> Pauli on boson qubits (padded)
##############################################################################

def embed_dense_to_qubits(H_dense):
    """
    Embed (d x d) dense block into (2^n x 2^n) by padding top-left.
    Returns padded matrix and n qubits.
    """
    d = H_dense.shape[0]
    n_qubits = int(ceil(log2(d)))
    dim_q = 2**n_qubits
    H_q = np.zeros((dim_q, dim_q), dtype=np.complex128)
    H_q[:d, :d] = H_dense
    return H_q, n_qubits

def pauli_decompose_dense(H_dense):
    """Dense boson block -> padded -> qml.Hamiltonian on boson wires 0..n_b-1."""
    H_q, n_b = embed_dense_to_qubits(H_dense)
    H_pl = qml.pauli_decompose(H_q)
    return H_pl, n_b

def strip_zero_terms(H, tol=1e-12):
    coeffs, ops = [], []
    for c, o in zip(H.coeffs, H.ops):
        if abs(c) > tol:
            coeffs.append(c)
            ops.append(o)
    return qml.Hamiltonian(coeffs, ops)

##############################################################################
# 4. Full SUSY QM Hamiltonian assembled in Pauli form (no full matrix)
##############################################################################

def build_susy_qm_hamiltonian(
    cutoff,
    potential,
    m=1.0,
    g=1.0,
    u=1.0,
    remove_zero_terms=True,
    fermion_wire=0
):
    """
    Build SUSY QM Hamiltonian as a qml.Hamiltonian without forming the full
    fermion⊗boson dense matrix.

    Wires:
      - fermion qubit on fermion_wire (default 0)
      - boson qubits on fermion_wire+1, ..., fermion_wire+n_b
    """
    # Bosonic dense blocks only
    A_b, B_b = bosonic_blocks(cutoff, potential, m=m, g=g, u=u)

    # Pauli-decompose boson blocks (on local boson wires 0..n_b-1)
    H_A_pl, n_b = pauli_decompose_dense(A_b)
    H_B_pl, _   = pauli_decompose_dense(B_b)

    # Shift boson wires -> global boson wires (fermion wire first)
    boson_wire_map = {w: fermion_wire + 1 + w for w in range(n_b)}

    full_coeffs = []
    full_ops = []

    # 1/2 * I_f ⊗ A_b
    for c, op in zip(H_A_pl.coeffs, H_A_pl.ops):
        op_b = qml.map_wires(op, boson_wire_map)
        full_ops.append(qml.prod(qml.Identity(fermion_wire), op_b))
        full_coeffs.append(0.5 * c)

    # 1/2 * Z ⊗ B_b
    for c, op in zip(H_B_pl.coeffs, H_B_pl.ops):
        op_b = qml.map_wires(op, boson_wire_map)
        full_ops.append(qml.prod(qml.PauliZ(fermion_wire), op_b))
        full_coeffs.append(0.5 * c)

    H_total = qml.Hamiltonian(full_coeffs, full_ops).simplify()
    if remove_zero_terms:
        H_total = strip_zero_terms(H_total)

    n_total_qubits = 1 + n_b
    return H_total, n_total_qubits


In [35]:
potential="DW"
cutoffs = [16]#,32,64,128]

dt=0.1
D=3
n_steps=1


path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\qml.expval\Adaptive-VQE\StatevectorFiles", potential,f"data_{16}.txt")

with open(path, "r", encoding="utf-8") as f:
            text = f.read()

json_part = text[: text.rfind("}") + 1]
data = json.loads(json_part)

params = []
for gate in data["circuit"]:
    p = gate["param"]
    params.append(p)


ansatze_name = f"CQAVQE_{potential}16_Reduced"

ansatz = ansatze.get(ansatze_name)
num_params = ansatz.n_params
max_gate = num_params + 1
pms = params[:max_gate]

all_data = []

for cutoff in cutoffs:

    print(f"Cutoff: {cutoff}")

    H_pauli, num_qubits = build_susy_qm_hamiltonian(cutoff=cutoff, potential=potential)
    H = qml.matrix(H_pauli, wire_order=list(range(num_qubits)))

    #H = calculate_Hamiltonian(cutoff, potential)
    #H_pauli = qml.pauli_decompose(H)

    t1 = datetime.now()
    eigenvalues, eigenvectors = np.linalg.eig(H)
    Ht = datetime.now() - t1

    min_index = np.argmin(eigenvalues)
    min_eigenvalue = eigenvalues[min_index]


    num_qubits = int(1+np.log2(cutoff))
    gates = ansatze.truncate_ansatz(ansatz, pms, num_qubits, max_gate=max_gate)

    dev = qml.device("default.qubit", wires=num_qubits, shots=4096)
    @qml.qnode(dev)
    def circuit(t, n_steps):

        #basis = [1] + [0]*(num_qubits-1)
        #qml.BasisState(basis, wires=list(range(num_qubits)))

        #for gate in gates:
        #        qml.apply(gate)

        qml.TrotterProduct(H_pauli.simplify(), time=t, n=n_steps)

        return qml.counts(wires=list(range(num_qubits)))

    

    times = [k * dt for k in range(D)] 
    all_counts = [circuit(t=t_k, n_steps=n_steps) for t_k in times]

    samples = Counter()
    for d in all_counts:
        samples.update(d)

    sorted_states = sorted(samples.items(), key=lambda x: x[1], reverse=True)
    top_states = [s for s, c in sorted_states]
    idx = [int(s, 2) for s in top_states]

    H_reduced = H[np.ix_(idx, idx)]

    t1 = datetime.now()
    eigenvalues, eigenvectors = np.linalg.eig(H_reduced)
    HRt = datetime.now() - t1

    mi = np.argmin(eigenvalues)
    me = eigenvalues[mi]

    row = {"potential": potential,
           "cutoff": cutoff,
           "dt": dt,
           "D": D,
           "n_steps": n_steps,
           "H_size": H.shape,
           "H_reduced_size": H_reduced.shape,
           "H_etime": str(Ht),
           "H_reduced_etime": str(HRt),
           "H_exact_e": min_eigenvalue.real,
           "H_reduced_e": me.real,
           "diff": np.abs(min_eigenvalue-me)
           }
    
    all_data.append(row)


Cutoff: 16


In [36]:
# Full matrix route
H_full = calculate_Hamiltonian(cutoff, potential)

# Projected-from-paulis route
pauli_terms = pauli_terms_from_operator(H_pauli, wire_order=list(range(num_qubits)))
H_red_proj = reduced_matrix_from_pauli_terms(pauli_terms, top_states)

# Slicing route
idx = [int(s, 2) for s in top_states]
H_red_slice = H_full[np.ix_(idx, idx)]

print("max diff:", np.max(np.abs(H_red_proj - H_red_slice)))


max diff: 2.842170943040401e-14


In [32]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,DW,16,0.1,3,1,"(32, 32)","(16, 16)",0:00:00.000267,0:00:00.000117,0.891599,0.891599,6.461498e-14
1,DW,32,0.1,3,1,"(64, 64)","(32, 32)",0:00:00.000927,0:00:00.000295,0.891632,0.891632,1.018075e-13
2,DW,64,0.1,3,1,"(128, 128)","(64, 64)",0:00:00.008086,0:00:00.001396,0.891632,0.891632,7.763790e-13
3,DW,128,0.1,3,1,"(256, 256)","(128, 128)",0:00:00.073632,0:00:00.031873,0.891632,0.891632,9.752199e-12


In [10]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,DW,16,0.1,3,1,"(32, 32)","(16, 16)",0:00:00.000280,0:00:00.000122,0.891599,0.891599,1.809664e-14
1,DW,32,0.1,3,1,"(64, 64)","(32, 32)",0:00:00.000888,0:00:00.000297,0.891632,0.891632,2.915446e-13
2,DW,64,0.1,3,1,"(128, 128)","(64, 64)",0:00:00.006416,0:00:00.001368,0.891632,0.891632,6.821210e-13
3,DW,128,0.1,3,1,"(256, 256)","(128, 128)",0:00:00.065502,0:00:00.034374,0.891632,0.891632,3.523293e-12


In [215]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\SBKQD\Files\NoInitial", potential)
os.makedirs(folder_path)
df.to_excel(os.path.join(folder_path, "SBKQD.xlsx"), index=False)

In [34]:
def apply_pauli_to_bitstring(pauli_str, bitstring):
    """
    Apply a Pauli string (like "IXYZ...") to a computational basis bitstring.

    Returns:
        phase (complex), out_bitstring (str)
    """
    phase = 1.0 + 0.0j
    out = list(bitstring)

    for q, (p, b_char) in enumerate(zip(pauli_str.upper(), bitstring)):
        b = 1 if b_char == "1" else 0

        if p == "I":
            continue
        elif p == "X":
            out[q] = "0" if b else "1"
        elif p == "Z":
            if b:
                phase *= -1
        elif p == "Y":
            out[q] = "0" if b else "1"
            phase *= (1j if b == 0 else -1j)  # i * (-1)^b
        else:
            raise ValueError(f"Bad Pauli char '{p}' at qubit {q}")

    return phase, "".join(out)


def reduced_matrix_from_pauli_terms(pauli_terms, basis_states):
    """
    Build reduced Hamiltonian matrix from explicit Pauli terms.

    pauli_terms: list of (coeff, pauli_str)
        e.g. [(0.5, "ZIIII"), (-1.2, "XXIYZ"), ...]
    basis_states: list of bitstrings, all same length n
        e.g. top_states from counts
    """
    basis_states = [s.strip() for s in basis_states]
    n = len(basis_states[0])
    m = len(basis_states)
    idx = {s: i for i, s in enumerate(basis_states)}

    H_red = np.zeros((m, m), dtype=complex)

    for coeff, pstr in pauli_terms:
        pstr = pstr.strip().upper()
        for ket in basis_states:
            phase, out_state = apply_pauli_to_bitstring(pstr, ket)
            if out_state in idx:
                i = idx[out_state]
                j = idx[ket]
                H_red[i, j] += coeff * phase

    return H_red


def op_to_full_pauli_string(op, wire_order):
    """
    Convert a single PennyLane Pauli word into a full string over wire_order.
    Example output: "IIXZY"
    """
    wire_pos = {w: i for i, w in enumerate(wire_order)}
    n = len(wire_order)
    chars = ["I"] * n

    def walk(o):
        # Products / tensors expose operands
        if hasattr(o, "operands") and o.operands is not None:
            for sub in o.operands:
                walk(sub)
            return

        name = o.name
        if name in ("PauliX", "X"):
            c = "X"
        elif name in ("PauliY", "Y"):
            c = "Y"
        elif name in ("PauliZ", "Z"):
            c = "Z"
        elif name in ("Identity", "I"):
            c = "I"
        else:
            raise ValueError(f"Unexpected operator in Pauli word: {name}")

        for w in o.wires:
            chars[wire_pos[w]] = c

    walk(op)
    return "".join(chars)


def pauli_terms_from_operator(H_pauli, wire_order):
    """
    Extract list of (coeff, pauli_str) from a PennyLane Hamiltonian / Sum.
    """
    if not hasattr(H_pauli, "terms"):
        raise TypeError("H_pauli has no .terms(); pass explicit pauli_terms instead.")

    coeffs, ops = H_pauli.terms()
    terms = []
    for c, o in zip(coeffs, ops):
        pstr = op_to_full_pauli_string(o, wire_order)
        terms.append((complex(c), pstr))
    return terms

